<a href="https://colab.research.google.com/github/WB-Jang/Training-Evaluation-Data-Set-Construction/blob/main/%ED%95%9C%EA%B5%AD%EC%9D%80%ED%96%89%EA%B2%BD%EC%A0%9C%EA%B8%88%EC%9C%B5%EC%9A%A9%EC%96%B4_Excel%EC%B6%94%EC%B6%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pdfplumber
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 33.4 MB/s eta 0:00:00


In [ ]:
import pdfplumber
import pandas as pd
import re

def pdf_to_excel(pdf_path, excel_path, start_page=18):
    print(f"📂 '{pdf_path}' 파일을 불러옵니다...")

    full_text = ""

    # 1. PDF 텍스트 추출
    with pdfplumber.open(pdf_path) as pdf:
        # 본문이 시작되는 페이지부터 끝까지 추출 (기본값 18페이지)
        # 페이지 인덱스는 0부터 시작하므로 start_page-1
        for i, page in enumerate(pdf.pages[start_page-1:], start=start_page):
            text = page.extract_text()
            if text:
                full_text += text + "\n"
            if i % 50 == 0:
                print(f"  - {i}페이지 읽는 중...")

    print("✅ 텍스트 추출 완료. 데이터 정리를 시작합니다.")

    # 2. 데이터 파싱 (용어, 설명, 연관검색어 분리)
    # '연관검색어'라는 키워드를 기준으로 텍스트를 나눕니다.
    chunks = re.split(r'연관검색어\s*[:：]', full_text)

    data = []

    # 첫 번째 덩어리 처리 (첫 용어와 설명)
    # 보통 페이지 헤더/푸터 등이 섞일 수 있어 정제가 필요할 수 있습니다.
    first_chunk_lines = [l.strip() for l in chunks[0].split('\n') if l.strip()]

    # 첫 용어 찾기 (간단한 로직: 가장 윗줄이 용어라고 가정)
    # 실제로는 헤더(경제금융용어 700선 등)를 제외해야 하므로 뒤에서부터 탐색
    if len(first_chunk_lines) >= 2:
        # 마지막 줄들이 설명이고, 그 앞이 용어일 가능성이 높음
        # 하지만 여기서는 단순화를 위해 첫 줄을 용어, 나머지를 설명으로 처리
        # (필요시 수동 보정 추천)
        current_term = first_chunk_lines[-2] if len(first_chunk_lines) > 1 else "미상"
        current_desc = first_chunk_lines[-1]
        current_entry = {'용어': current_term, '설명': current_desc}
    else:
        current_entry = None

    # 중간 및 마지막 덩어리 처리
    for chunk in chunks[1:]:
        lines = [l.strip() for l in chunk.split('\n') if l.strip()]
        if not lines: continue

        # 1. 이전 항목의 '연관검색어' 가져오기 (청크의 첫 줄)
        related_terms = lines[0]
        if current_entry:
            current_entry['연관검색어'] = related_terms
            data.append(current_entry)

        # 2. 다음 항목의 '용어'와 '설명' 찾기
        # 다음 용어는 보통 연관검색어 다음 줄이나 다다음 줄에 위치
        if len(lines) > 1:
            # 헤더(페이지 번호 등) 제거 로직: 숫자로만 된 줄이나 책 제목 등은 건너뜀
            content_lines = [l for l in lines[1:] if not l.isdigit() and "경제금융용어" not in l]

            if content_lines:
                new_term = content_lines[0] # 첫 줄을 용어로 간주
                new_desc = " ".join(content_lines[1:]) # 나머지는 설명
                current_entry = {'용어': new_term, '설명': new_desc}
            else:
                current_entry = None
        else:
            current_entry = None

    # 3. 엑셀 저장
    if data:
        df = pd.DataFrame(data)
        # 데이터 정제 (줄바꿈 문자 등 제거)
        df['용어'] = df['용어'].str.replace(r'[ㄱ-ㅎ]\s*$', '', regex=True) # 섹션 자음(ㄱ,ㄴ..) 제거

        df.to_excel(excel_path, index=False)
        print(f"🎉 변환 성공! '{excel_path}' 파일이 생성되었습니다.")
        print(f"   총 {len(df)}개의 용어가 추출되었습니다.")
    else:
        print("⚠️ 데이터를 찾지 못했습니다. 페이지 범위를 확인해주세요.")

# 실행 (파일명이 다르면 수정해주세요)
pdf_filename = "2020_경제금융용어 700선_게시.pdf"
excel_filename = "경제금융용어사전_정리.xlsx"

try:
    pdf_to_excel(pdf_filename, excel_filename)
except FileNotFoundError:
    print(f"❌ 파일을 찾을 수 없습니다. '{pdf_filename}' 파일이 같은 폴더에 있는지 확인해주세요.")

📂 '2020_경제금융용어 700선_게시.pdf' 파일을 불러옵니다...
  - 50페이지 읽는 중...
  - 100페이지 읽는 중...
  - 150페이지 읽는 중...
  - 200페이지 읽는 중...
  - 250페이지 읽는 중...
  - 300페이지 읽는 중...
  - 350페이지 읽는 중...
✅ 텍스트 추출 완료. 데이터 정리를 시작합니다.
🎉 변환 성공! '경제금융용어사전_정리.xlsx' 파일이 생성되었습니다.
   총 547개의 용어가 추출되었습니다.
